Reference: 
- [1] https://langchain-ai.github.io/langgraph/tutorials/rewoo/rewoo/
- [2] Reasoning WithOut Observation(ReWOO): https://arxiv.org/abs/2305.18323

In [1]:
# State
from typing import List, TypedDict


class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

In [3]:
prompt = """For the following task, make plans that can solve the problem step by step. For each plan, indicate \
which external tool together with tool input to retrieve evidence. You can store the evidence into a \
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Google[input]: Worker that searches results from Google. Useful when you need to find short
and succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general
world knowledge and common sense. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

For example,
Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked x
hours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours
less than Toby. How many hours did Rebecca work?
Plan: Given Thomas worked x hours, translate the problem into algebraic expressions and solve
with Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x − 10) + ((2x − 10) − 8) = 157]
Plan: Find out the number of hours Thomas worked. #E2 = LLM[What is x, given #E1]
Plan: Calculate the number of hours Rebecca worked. #E3 = Calculator[(2 ∗ #E2 − 10) − 8]

Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.

Task: {task}"""

In [4]:
task = "what is the hometown of the 2024 australian open winner"

In [5]:
result = model.invoke(prompt.format(task=task))

In [6]:
print(result.content)

Plan: Use Google to search for the hometown of the 2024 Australian Open winner. #E1 = Google[2024 Australian Open winner hometown]
Plan: Retrieve the information about the hometown of the 2024 Australian Open winner from the search results. #E2 = LLM[What is the hometown of the 2024 Australian Open winner, given #E1]


In [43]:
import re

from langchain_core.prompts import ChatPromptTemplate

# Regex to match expressions of the form E#... = ...[...]
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*([^([^([^]+)\]"
prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
planner = prompt_template | model


def get_plan(state: ReWOO):
    task = state["task"]
    result = planner.invoke({"task": task})
    print(">>> result:", result)
    # Find all matches in the sample text
    matches = re.findall(regex_pattern, result.content)
    print(">>> matchs:", matches)
    print(">>> content:", result.content)
    print(">>>> end")
    return {"steps": matches, "plan_string": result.content}

In [44]:
# Executor
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [45]:
def _get_current_task(state: ReWOO):
    print(">>>", state["results"])
    if state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1


def tool_execution(state: ReWOO):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    print(">>>> _step:", _step)
    print(">>>> steps:", state["steps"])
    _, step_name, tool, tool_input = state["steps"][_step - 1]
    _results = state["results"] or {}
    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)
    if tool == "Google":
        result = search.invoke(tool_input)
    elif tool == "LLM":
        result = model.invoke(tool_input)
    else:
        raise ValueError
    _results[step_name] = str(result)
    return {"results": _results}

In [46]:
# Solver
solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer
directly with no extra words.

Task: {task}
Response:"""


def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = state["results"] or {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {tool}[{tool_input}]"
    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = model.invoke(prompt)
    return {"result": result.content}

In [47]:
# define graph
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        # We have executed all tasks
        return "solve"
    else:
        # We are still executing tasks, loop back to the "tool" node
        return "tool"

In [48]:
from langgraph.graph import END, StateGraph, START

graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)
graph.add_conditional_edges("tool", _route)
graph.add_edge(START, "plan")

app = graph.compile()

In [49]:
for s in app.stream({"task": task}):
    print(s)
    print("---")

>>> result: content='Plan: Use Google to search for the hometown of the 2024 Australian Open winner. #E1 = Google[2024 Australian Open winner hometown] \nPlan: Review the search results to find the hometown of the 2024 Australian Open winner. #E2 = LLM[Review search results from #E1] \nPlan: Confirm the hometown of the 2024 Australian Open winner. #E3 = LLM[What is the hometown of the 2024 Australian Open winner, given #E2]' response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 377, 'total_tokens': 483}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-a87d26e0-a5ab-463e-9415-254930783194-0' usage_metadata={'input_tokens': 377, 'output_tokens': 106, 'total_tokens': 483}
>>> matchs: []
>>> content: Plan: Use Google to search for the hometown of the 2024 Australian Open winner. #E1 = Google[2024 Australian Open winner hometown] 
Plan: Review the search results to find the hometown of the 2024 Australian 

IndexError: list index out of range